# Samoa 5m DEM creation
Load in LiDAR where it exists and combine with FABDEM to produce 5m DEMs

In [ ]:
%load_ext autoreload
%autoreload 2

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import pathlib

import os
import sys
module_path = os.path.abspath(os.path.join("..", "src", "pacific-dems"))
if module_path not in sys.path:
    sys.path.append(module_path)

import create_dem_functions
import islands
import create_data_paths

# Define country and resolution

In [ ]:
resolution = 5 # in meters
country_name = "samoa"

# Create paths and get contry information

In [ ]:
paths = create_data_paths.get_paths(country_name=country_name, resolution=resolution)
islands_dict = islands.get_island(country_name)

# Specify any LiDAR data

In [ ]:
# Path of LiDAR DEMs
savaii_lidar = create_dem_functions.load_dem(paths["lidar"] / "Savaii_Seamless_5m_DEM.tif")
upolu_lidar = create_dem_functions.load_dem(paths["lidar"] / "Upolu_Seamless_5m_DEM.tif")

# Add to islands
islands_dict["savai'i"]["lidar"] = savaii_lidar
islands_dict["savai'i"]["lidar_only"] = True
islands_dict["upolu"]["lidar"] = upolu_lidar
islands_dict["upolu"]["lidar_only"] = True

# Get island outlines, combine FABDEMs and create DEM# Get island outlines and combine FABDEMs

In [ ]:
for island_name, island_dict in islands_dict.items():
    print(f"Setup for {island_name}")
    island_dict["name"] = island_name
    island_dict["land"] = create_dem_functions.get_islands_in_boundary(island_dict=island_dict, gadm_path=paths["land"], output_path=paths["output"] / island_name)
    island_dict["fab"] = create_dem_functions.combine_fabs_in_boundary(island_dict=island_dict, fab_path=paths["fabdem"])
create_dem_functions.creating_dems_all_islands(island_groups=islands_dict, resolution=resolution, output_path=paths["output"])